In [10]:
#random forest  DONE!!
import pandas as pd
import datetime 
import numpy as np
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score, classification_report,confusion_matrix  #to be used in statistics and also accuracy
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler,scale
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
#sns.set(style=whitegrid, color_codes=True)
import statsmodels.api as sm

data = pd.read_excel(r'Dropbox/Project personal(400)/DATA FOR COMPUTER SCIENCE STUDENT NEW.xls',sheet_name='INACTIVE EMPLOYEES',  parse_date={'Expire Date'})

#eliminating attributes not needed
data = data.drop(['Personnel Number','Country Name','Birth Date','Age.1'],axis=1)
print(data.shape)
print(data.head())
data.dtypes


(4329, 9)
  Gnd Marital Status  Age Ethnic Group Name           Post Name  \
0   M              S   60     GREATER ACCRA     SENIOR LECTURER   
1   M              S   61           CENTRAL     SENIOR LECTURER   
2   M              M   53           ASHANTI     SENIOR LECTURER   
3   F              S   39           EASTERN            LECTURER   
4   M              S   36           EASTERN  PART-TIME LECTURER   

               Appt. Type                 Department Name Expire Date  \
0  SENIOR MEMBER ACADEMIC                MODERN LANGUAGES  2005-08-01   
1  SENIOR MEMBER ACADEMIC                MODERN LANGUAGES  1997-10-01   
2  SENIOR MEMBER ACADEMIC  DEPT. OF NUTRITION & FOOD SCI.  2016-01-01   
3  SENIOR MEMBER ACADEMIC    POP.FAMILY & REPR.HEALTH-SPH  2006-08-01   
4  SENIOR MEMBER ACADEMIC                           ISSER  2002-08-01   

  Reason for Separation  
0            RETIREMENT  
1            RETIREMENT  
2  VOLUNTARY RETIREMENT  
3              RESIGNED  
4      VACATION OF

Gnd                              object
Marital Status                   object
Age                               int64
Ethnic Group Name                object
Post Name                        object
Appt. Type                       object
Department Name                  object
Expire Date              datetime64[ns]
Reason for Separation            object
dtype: object

In [12]:
#finding the correlation
data.count()
data.groupby('Reason for Separation').mean()
#from the meann grouping below, it is evident that most employees retired due to their age
#followed by those who requested for an early retirement before voluntary retirement
#conclusion: this shows that most employees from the University retired as compared to the other factors under reason of termination.

,Age
Reason for Separation,
DECEASED,49.220339
DISMISSED,36.051282
EARLY RETIREMENT,56.000000
END OF APPOINTMENT,41.657895
NON-CONFIRMATION,42.800000
RESIGNED,35.477573
RETIREMENT,60.241641
SEPARATION,36.300000
TERMINATED,40.125000


In [13]:
#data['Expire Data'] = data(['Expire Data'].dtype(np.int64),axis=1)
numeric_data = list(data.dtypes[data.dtypes != "object"].index)
X1 = data[numeric_data]
print(X1.head())
#data.dtypes["Expire Date"] = data.dtypes(Timestamp)
X1.shape

   Age Expire Date
0   60  2005-08-01
1   61  1997-10-01
2   53  2016-01-01
3   39  2006-08-01
4   36  2002-08-01


(4329, 2)

In [5]:
data.dtypes

Gnd                              object
Marital Status                   object
Age                               int64
Ethnic Group Name                object
Post Name                        object
Appt. Type                       object
Department Name                  object
Expire Date              datetime64[ns]
Reason for Separation            object
dtype: object

In [14]:
data['Expire Date'] = pd.to_datetime(data['Expire Date'])
data['year'], data['month'], data['day'] = data['Expire Date'].dt.year, data['Expire Date'].dt.month, data['Expire Date'].dt.day
data.head()

,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appt. Type,Department Name,Expire Date,Reason for Separation,year,month,day
0,M,S,60,GREATER ACCRA,SENIOR LECTURER,SENIOR MEMBER ACADEMIC,MODERN LANGUAGES,2005-08-01,RETIREMENT,2005,8,1
1,M,S,61,CENTRAL,SENIOR LECTURER,SENIOR MEMBER ACADEMIC,MODERN LANGUAGES,1997-10-01,RETIREMENT,1997,10,1
2,M,M,53,ASHANTI,SENIOR LECTURER,SENIOR MEMBER ACADEMIC,DEPT. OF NUTRITION & FOOD SCI.,2016-01-01,VOLUNTARY RETIREMENT,2016,1,1
3,F,S,39,EASTERN,LECTURER,SENIOR MEMBER ACADEMIC,POP.FAMILY & REPR.HEALTH-SPH,2006-08-01,RESIGNED,2006,8,1
4,M,S,36,EASTERN,PART-TIME LECTURER,SENIOR MEMBER ACADEMIC,ISSER,2002-08-01,VACATION OF POST,2002,8,1


In [16]:
data = data.drop(['Expire Date'], axis=1)
#data.dtypes['Expire Date'] = pd.to_datetime(data['Expire Date'])
data.dtypes

KeyError: "['Expire Date'] not found in axis"

In [8]:
numeric_data = list(data.dtypes[data.dtypes != "object"].index)
X1 = data[numeric_data]
print(X1.head())
X1.shape

   Age  year  month  day
0   60  2005      8    1
1   61  1997     10    1
2   53  2016      1    1
3   39  2006      8    1
4   36  2002      8    1


(4329, 4)

In [6]:
data.groupby('Reason for Separation').count()
#from the count results below,  its evident that employees who retired are the most in the dataset , followed by those who resigned.
#vacation of post also gave the third highest number of employees in from the dataset.
# The least number of employees from the reason for separation was for those that were non-confirmed
#All other forms of retirement(early retirement and voluntary retirement) will be classified under retirement.

,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appt. Type,Department Name,year,month,day
Reason for Separation,,,,,,,,,,
DECEASED,413,413,413,413,413,413,413,413,413,413
DISMISSED,39,39,39,39,39,39,39,39,39,39
EARLY RETIREMENT,1,1,1,1,1,1,1,1,1,1
END OF APPOINTMENT,76,76,76,76,76,76,76,76,76,76
NON-CONFIRMATION,5,5,5,5,5,5,5,5,5,5
RESIGNED,758,758,758,758,758,758,758,758,758,758
RETIREMENT,1974,1974,1974,1974,1974,1974,1974,1974,1974,1974
SEPARATION,40,40,40,40,40,40,40,40,40,40
TERMINATED,160,160,160,160,160,160,160,160,160,160


In [ ]:
VOLUNTARY TERMINATION
INVOLUNTARY TERMINATIO
DECEASED
DISMISSED
RESIGNED
SEPARATION,VACATION OF POST IS ADDED TO RESIGNED
EARLY AND VOLUNTARY RETIREMENT ADDED TO RETIREMENT
DISMISSED ,NON-CONFIRMATION, DECEASED, END OF APPOINTMENT  IS ADDED TO TERMINATED
 IS ADDED TO 

In [7]:
#Classification of Reason for Separation
data['Reason for Separation']=np.where(data['Reason for Separation']=='EARLY RETIREMENT','RETIREMENT',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='VOLUNTARY RETIREMENT','RETIREMENT',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='SEPARATION','RESIGNED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='VACATION OF POST','RESIGNED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='DISMISSED','TERMINATED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='NON-CONFIRMATION','TERMINATED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='DECEASED','TERMINATED',data['Reason for Separation'])
data['Reason for Separation']=np.where(data['Reason for Separation']=='END OF APPOINTMENT','TERMINATED',data['Reason for Separation'])
data.groupby('Reason for Separation').count()



,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appt. Type,Department Name,year,month,day
Reason for Separation,,,,,,,,,,
RESIGNED,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459
RETIREMENT,2177,2177,2177,2177,2177,2177,2177,2177,2177,2177
TERMINATED,693,693,693,693,693,693,693,693,693,693


In [8]:
#isolating the target variable
Y_raw = data.pop("Reason for Separation")
#creating dummmy variables of target
Y = pd.get_dummies(Y_raw)
print(Y.head())
Y.shape
#info()

   RESIGNED  RETIREMENT  TERMINATED
0         0           1           0
1         0           1           0
2         0           1           0
3         1           0           0
4         1           0           0


(4329, 3)

In [122]:
index_col='Expire Date'

In [9]:
X1_train,X1_test,Y_train,Y_test=train_test_split(X1,Y,test_size=0.2,random_state=42)
model = RandomForestClassifier()
model.fit(X1_train,Y_train)
model_predictions = model.predict(X1_test)
print("Accuracy : ", accuracy_score(Y_test,model_predictions))

C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


TypeError: float() argument must be a string or a number, not 'Timestamp'

In [13]:
print(X1_test.shape)
print(X1_train.shape)
print(Y_test.shape)
print(Y_train.shape)

(866, 4)
(3463, 4)
(866, 3)
(3463, 3)


In [14]:
categorical_data = list(data.dtypes[data.dtypes == "object"].index)
#creating dummy variables.
X2 = pd.get_dummies(data[categorical_data])
print(X2.head())


   Gnd_F  Gnd_M  Marital Status_D  Marital Status_M  Marital Status_S  \
0      0      1                 0                 0                 1   
1      0      1                 0                 0                 1   
2      0      1                 0                 1                 0   
3      1      0                 0                 0                 1   
4      0      1                 0                 0                 1   

   Marital Status_W  Ethnic Group Name_AMERICAS  Ethnic Group Name_ASHANTI  \
0                 0                           0                          0   
1                 0                           0                          0   
2                 0                           0                          1   
3                 0                           0                          0   
4                 0                           0                          0   

   Ethnic Group Name_ASIA  Ethnic Group Name_BRONG AHAFO  ...  \
0                       0  

In [15]:
X = pd.concat([X1,X2],axis = 1)
X.head()

,Age,year,month,day,Gnd_F,Gnd_M,Marital Status_D,Marital Status_M,Marital Status_S,Marital Status_W,...,Department Name_UNIVERSITY CONSULTANCY CENTRE,Department Name_UNIVERSITY HOSPITAL,Department Name_VC'S OFFICE,Department Name_VICE-CHANCELLORS GHANA,Department Name_VIROLOGY UNIT,Department Name_VOLTA HALL,Department Name_VRBRP,Department Name_WACCI,Department Name_WATER DRAINAGE & SEW. SECTION,Department Name_WORKS SECTION
0,60,2005,8,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,61,1997,10,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,53,2016,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,39,2006,8,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,36,2002,8,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train,Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
#save the model to disk
filename = 'finalizedUG_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [18]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test,Y_test)
print(result)

0.8106235565819861


In [19]:
#OTHER OPTIONS OF FINDING THE METRICS 
model_predictions = model.predict(X_test)

count_misclassified = (Y_test != model_predictions).sum()
print('Misclassified samples: {}'.format(count_misclassified))
print(classification_report(Y_test,model_predictions))
print("Accuracy : ", accuracy_score(Y_test,model_predictions))
print ("c-stat:", roc_auc_score(Y_test,model_predictions))
print(confusion_matrix(Y_test.values.argmax(axis=1),model_predictions.argmax(axis=1)))

Misclassified samples: RESIGNED      100
RETIREMENT     45
TERMINATED    114
dtype: int64
              precision    recall  f1-score   support

           0       0.84      0.82      0.83       291
           1       0.96      0.94      0.95       454
           2       0.55      0.31      0.40       121

   micro avg       0.88      0.81      0.84       866
   macro avg       0.78      0.69      0.73       866
weighted avg       0.86      0.81      0.83       866
 samples avg       0.81      0.81      0.81       866

Accuracy :  0.8106235565819861
c-stat: 0.8176054981936168
[[260   6  25]
 [ 22 426   6]
 [ 72  11  38]]


C:\Users\Anna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [20]:
data_new = pd.read_excel(r'Dropbox/Project personal(400)/DATA FOR COMPUTER SCIENCE STUDENT NEW.xls')

#eliminating attributes not needed
data_new = data_new.drop(['No.','Country Name'],axis=1)
print(data_new.shape)
print(data_new.head())
data_new.dtypes

(6139, 9)
  Gnd Marital Status  Age Ethnic Group Name Appointment Date  \
0   F         Single   46           EASTERN       2003-05-02   
1   M         Single   37           ASHANTI       2018-03-02   
2   M        Married   32        UPPER WEST       2018-02-08   
3   M         Single   36           EASTERN       2012-11-01   
4   M        Married   36           EASTERN       2018-02-06   

          Post Name  Appointment Names     Department Name  Annual Basic  
0  ASSISTANT PORTER  JS: NON-TECHNICAL  A. A. KWAPONG HALL         16727  
1  ASSISTANT PORTER  JS: NON-TECHNICAL  A. A. KWAPONG HALL         15637  
2  ASSISTANT PORTER  JS: NON-TECHNICAL  A. A. KWAPONG HALL         15637  
3  ASSISTANT PORTER  JS: NON-TECHNICAL  A. A. KWAPONG HALL         15637  
4  ASSISTANT PORTER  JS: NON-TECHNICAL  A. A. KWAPONG HALL         15637  


Gnd                          object
Marital Status               object
Age                           int64
Ethnic Group Name            object
Appointment Date     datetime64[ns]
Post Name                    object
Appointment Names            object
Department Name              object
Annual Basic                  int64
dtype: object

In [21]:
data_new.groupby('Appointment Names').count()

,Gnd,Marital Status,Age,Ethnic Group Name,Appointment Date,Post Name,Department Name,Annual Basic
Appointment Names,,,,,,,,
JS: NON-TECHNICAL,2627,2626,2627,2627,2627,2627,2627,2627
JS: TECHNICAL,2,2,2,2,2,2,2,2
JUNIOR STAFF TEMP.,7,7,7,7,7,7,7,7
SM ADMIN/PROF,261,261,261,261,261,261,261,261
SM RESEARCH,1,1,1,1,1,1,1,1
SM TEACHING,1199,1199,1199,1199,1199,1199,1199,1199
SMT-PENS: CONTRACT,162,162,162,162,162,162,162,162
SS-PENSIONER CONT.,1,1,1,1,1,1,1,1
SS: CONTRACT TECH,1,1,1,1,1,1,1,1


In [22]:
#Classification of Reason for Separation
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='JS: NON-TECHNICAL','JS: TECHNICAL',data_new['Appointment Names'])
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='SS: CONTRACT TECH','SS: NON-TECHNICAL',data_new['Appointment Names'])
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='SS-PENSIONER CONT.','SS: NON-TECHNICAL',data_new['Appointment Names'])
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='SS: TECHNICAL','SS: NON-TECHNICAL',data_new['Appointment Names'])

data_new['Appointment Names']=np.where(data_new['Appointment Names']=='SMT-PENS: CONTRACT','SM ADMIN/PROF',data_new['Appointment Names'])
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='SM RESEARCH','SM ADMIN/PROF',data_new['Appointment Names'])
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='SM TEACHING','SM ADMIN/PROF',data_new['Appointment Names'])
data_new['Appointment Names']=np.where(data_new['Appointment Names']=='VISITING SCHOLAR','SM ADMIN/PROF',data_new['Appointment Names'])

data_new['Appointment Names']=np.where(data_new['Appointment Names']=='TEACHING ASSISTANT','JUNIOR STAFF TEMP.',data_new['Appointment Names'])

data_new.groupby('Appointment Names').count()


,Gnd,Marital Status,Age,Ethnic Group Name,Appointment Date,Post Name,Department Name,Annual Basic
Appointment Names,,,,,,,,
JS: TECHNICAL,2629,2628,2629,2629,2629,2629,2629,2629
JUNIOR STAFF TEMP.,20,20,20,20,20,20,20,20
SM ADMIN/PROF,1643,1643,1643,1643,1643,1643,1643,1643
SS: NON-TECHNICAL,1847,1847,1847,1847,1847,1847,1847,1847


In [23]:
data_new['Appointment Date'] = pd.to_datetime(data_new['Appointment Date'])
data_new['year'], data_new['month'],data_new['day'] = data_new['Appointment Date'].dt.year, data_new['Appointment Date'].dt.month, data_new['Appointment Date'].dt.day
data_new.head()

,Gnd,Marital Status,Age,Ethnic Group Name,Appointment Date,Post Name,Appointment Names,Department Name,Annual Basic,year,month,day
0,F,Single,46,EASTERN,2003-05-02,ASSISTANT PORTER,JS: TECHNICAL,A. A. KWAPONG HALL,16727,2003,5,2
1,M,Single,37,ASHANTI,2018-03-02,ASSISTANT PORTER,JS: TECHNICAL,A. A. KWAPONG HALL,15637,2018,3,2
2,M,Married,32,UPPER WEST,2018-02-08,ASSISTANT PORTER,JS: TECHNICAL,A. A. KWAPONG HALL,15637,2018,2,8
3,M,Single,36,EASTERN,2012-11-01,ASSISTANT PORTER,JS: TECHNICAL,A. A. KWAPONG HALL,15637,2012,11,1
4,M,Married,36,EASTERN,2018-02-06,ASSISTANT PORTER,JS: TECHNICAL,A. A. KWAPONG HALL,15637,2018,2,6


In [24]:
data_new = data_new.drop(['Appointment Date'], axis=1)
#data.dtypes['Expire Date'] = pd.to_datetime(data['Expire Date'])
data_new.dtypes

Gnd                  object
Marital Status       object
Age                   int64
Ethnic Group Name    object
Post Name            object
Appointment Names    object
Department Name      object
Annual Basic          int64
year                  int64
month                 int64
day                   int64
dtype: object

In [25]:
data_new.groupby('Department Name').count()


,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appointment Names,Annual Basic,year,month,day
Department Name,,,,,,,,,,
A. A. KWAPONG HALL,17,17,17,17,17,17,17,17,17,17
ABIBIGOROMMA,25,25,25,25,25,25,25,25,25,25
ACADEMIC AFFAIRS,65,64,65,65,65,65,65,65,65,65
ACADEMIC QUALITY ASSURANCE,6,6,6,6,6,6,6,6,6,6
ACCRA CITY CAMPUS,43,43,43,43,43,43,43,43,43,43
ACCRA LEARNING CENTRE,11,11,11,11,11,11,11,11,11,11
ADULT HEALTH - SCH OF NURSING,8,8,8,8,8,8,8,8,8,8
AKUAFO HALL,77,77,77,77,77,77,77,77,77,77
AWUDOME-TSITO LEARNING CENTRE,10,10,10,10,10,10,10,10,10,10


In [27]:
#Classification of Department Name
data_new['Department Name']=np.where(data_new['Department Name']=='A. A. KWAPONG HALL','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='VOLTA HALL','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='AKUAFO HALL','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='COMMONWEALTH','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='MENSAH SARBAH HALL','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='JUBILEE HALL/ISH','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='HILLA LIMANN HALL','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='E. FRANCES SEY HALL','LEGON HALL',data_new['Department Name'])
data_new['Department Name']=np.where(data_new['Department Name']=='JEAN NELSON AKA HALL','LEGON HALL',data_new['Department Name'])


data_new.groupby('Department Name').count()



,Gnd,Marital Status,Age,Ethnic Group Name,Post Name,Appointment Names,Annual Basic,year,month,day
Department Name,,,,,,,,,,
ABIBIGOROMMA,25,25,25,25,25,25,25,25,25,25
ACADEMIC AFFAIRS,65,64,65,65,65,65,65,65,65,65
ACADEMIC QUALITY ASSURANCE,6,6,6,6,6,6,6,6,6,6
ACCRA CITY CAMPUS,43,43,43,43,43,43,43,43,43,43
ACCRA LEARNING CENTRE,11,11,11,11,11,11,11,11,11,11
ADULT HEALTH - SCH OF NURSING,8,8,8,8,8,8,8,8,8,8
AWUDOME-TSITO LEARNING CENTRE,10,10,10,10,10,10,10,10,10,10
BACTERIOLOGY UNIT,16,16,16,16,16,16,16,16,16,16
BALME : ACQUISITION,4,4,4,4,4,4,4,4,4,4


In [28]:

data_new.groupby('Post Name').count()

,Gnd,Marital Status,Age,Ethnic Group Name,Appointment Names,Department Name,Annual Basic,year,month,day
Post Name,,,,,,,,,,
HALL ASSISTANT,2,2,2,2,2,2,2,2,2,2
ACCOUNTANT,20,20,20,20,20,20,20,20,20,20
ACCOUNTING ASSISTANT,1,1,1,1,1,1,1,1,1,1
ACCOUNTING ASSISTANT,53,53,53,53,53,53,53,53,53,53
ACCOUNTS CLERK GRADE I,6,6,6,6,6,6,6,6,6,6
ACCOUNTS CLERK GRADE II,2,2,2,2,2,2,2,2,2,2
ACTING DIRECTOR,1,1,1,1,1,1,1,1,1,1
ADJUNCT SCHOLAR,1,1,1,1,1,1,1,1,1,1
ADMINISTRATIVE ASSISTANT,1,1,1,1,1,1,1,1,1,1


In [52]:

#Classification of Post Name
#ACCOUNTING ASSISTANT
#data_new['Post Name']=np.where(data_new['Post Name']=='ACCOUNTING ASSISTANT','ACCOUNTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ACCOUNTS CLERK GRADE II','ACCOUNTS CLERK GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ACCOUNTING ASSISTANT','ACCOUNTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ACCOUNTS CLERK GRADE II','ACCOUNTS CLERK GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ADMINISTRATIVE ASSISTANT','ADMINISTRATIVE ASSISTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ARTISAN ( ELECTRICIAN)','ARTISAN',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ARTISAN ( PLUMBER)','ARTISAN',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ARTISAN-GRADUATE HOSTEL','ARTISAN',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='ASSISTANT DESIGNER GRADE II','ASSISTANT DESIGNER GRADE I',data_new['Post Name'])

data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( CARPENTER)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( CARPENTRY)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( ELECTRICIAN)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( PLUMBER)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( REFRIG & AIR COND)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( MASON)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( PAINTER)','TRADESMAN GRADE I',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE I ( REFRIGE. MECHANIC)','TRADESMAN GRADE I',data_new['Post Name'])




data_new['Post Name']=np.where(data_new['Post Name']=='TECHNICAL ASSISTANT GD I','TECHNICAL ASSISTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TECHNICAL ASSISTANT GD II','TECHNICAL ASSISTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TECHNICAL ASSISTANT GRADE I','TECHNICAL ASSISTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TECHNICAL ASSISTANT GRADE II','TECHNICAL ASSISTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TECHNICAL ASSISTANT GRADE I (X-RAY)','TECHNICAL ASSISTANT',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TTECHNICAL ASSISTANT GRADE II','TECHNICAL ASSISTANT',data_new['Post Name'])

data_new['Post Name']=np.where(data_new['Post Name']=='TELEPHONE EXCHANGE SUPERVISOR','TELEPHONIST',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TELEPHONE SUPERVISOR','TELEPHONIST',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRACTOR OPERATOR GRADE I','TRACTOR OPERATOR',data_new['Post Name'])

#data_new['Post Name']=np.where(data_new['Post Name']=='TECHNICAL ASSISTANT GD I','TELEPHONIST',data_new['Post Name'])

data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II ( TAILOR)','TRADESMAN GRADE II',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II (CARPENTER)','TRADESMAN GRADE II',data_new['Post Name'])


data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II (ELECTRICAL)','TRADESMAN GRADE II',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II (ELECTRONICS)','TRADESMAN GRADE II',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II (LIFEGUARD)','TRADESMAN GRADE II',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II (REFRIGERATION UNIT)','TRADESMAN GRADE II',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TRADESWOMAN GRADE II ( PLUMBER)','TRADESMAN GRADE II',data_new['Post Name'])
data_new['Post Name']=np.where(data_new['Post Name']=='TYPIST GRADE II','TYPIST GRADE I',data_new['Post Name'])

data_new['Post Name']=np.where(data_new['Post Name']=='TRADESMAN GRADE II (CARPENTER)','TRADESMAN GRADE II',data_new['Post Name'])
data_new.groupby('Post Name').count()

,Gnd,Marital Status,Age,Ethnic Group Name,Appointment Names,Department Name,Annual Basic,year,month,day
Post Name,,,,,,,,,,
HALL ASSISTANT,2,2,2,2,2,2,2,2,2,2
ACCOUNTANT,73,73,73,73,73,73,73,73,73,73
ACCOUNTING ASSISTANT,1,1,1,1,1,1,1,1,1,1
ACCOUNTS CLERK GRADE I,8,8,8,8,8,8,8,8,8,8
ACTING DIRECTOR,1,1,1,1,1,1,1,1,1,1
ADJUNCT SCHOLAR,1,1,1,1,1,1,1,1,1,1
ADMINISTRATIVE ASSISTANT,1,1,1,1,1,1,1,1,1,1
ADMINISTRATIVE ASSISTANT,100,100,100,100,100,100,100,100,100,100
ADMINISTRATIVE ASSISTANT (ARCHIVES),2,2,2,2,2,2,2,2,2,2


In [48]:
numeric_data = list(data_new.dtypes[data_new.dtypes != "object"].index)
X1 = data_new[numeric_data]
print(X1.head())

   Age  Annual Basic  year  month  day
0   46         16727  2003      5    2
1   37         15637  2018      3    2
2   32         15637  2018      2    8
3   36         15637  2012     11    1
4   36         15637  2018      2    6


In [49]:
categorical_data = list(data_new.dtypes[data_new.dtypes == "object"].index)
#creating dummy variables.
X2 = pd.get_dummies(data_new[categorical_data])
print(X2.head())


   Gnd_F  Gnd_M  Marital Status_Divorced  Marital Status_Married  \
0      1      0                        0                       0   
1      0      1                        0                       0   
2      0      1                        0                       1   
3      0      1                        0                       0   
4      0      1                        0                       1   

   Marital Status_Single  Marital Status_Widow(er)  \
0                      1                         0   
1                      1                         0   
2                      0                         0   
3                      1                         0   
4                      0                         0   

   Ethnic Group Name_AMERICAS  Ethnic Group Name_ASHANTI  \
0                           0                          0   
1                           0                          1   
2                           0                          0   
3                         

In [50]:
X_new = pd.concat([X1,X2],axis = 1)
X_new.head()

,Age,Annual Basic,year,month,day,Gnd_F,Gnd_M,Marital Status_Divorced,Marital Status_Married,Marital Status_Single,...,Department Name_UNIV. OF GHANA PRINTING PRESS,Department Name_UNIVERSITY BASIC SCHOOLS,Department Name_UNIVERSITY HOSPITAL,Department Name_VC'S OFFICE,Department Name_VICE-CHANCELLORS GHANA,Department Name_VIROLOGY UNIT,Department Name_WACCBIP,Department Name_WACCI,Department Name_WATER DRAINAGE & SEW. SECTION,Department Name_WORKS SECTION
0,46,16727,2003,5,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,37,15637,2018,3,2,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,32,15637,2018,2,8,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,36,15637,2012,11,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,36,15637,2018,2,6,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
#X_new, _ = make_blobs(n_samples)
Y_new = model.predict(X_new)

ValueError: Number of features of the model must match the input. Model n_features is 649 and input n_features is 692 